[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/06-YUV_compression/color-DCT_compression.ipynb)

# Color-DCT image compression

Removing redundancy in the color domain with the [DCT](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.dct.html).

The DCT is orthonormal (orthongonal + unitary (energy preserving transform)), and as it happens with other energy compacting transforms, the gain of some subbands (color-DCT components in our case) will be higher (the coefficients in such subbands will be larger) compared to other subbands. Because the quantization error generated by an uniform quantizer is independent of the signal amplitude (the value of the color-DCT components), the SNR will be higher in those subbands with a higher gain.

Moreover, as a consequence of that the number of high energy coefficients tends to be small (depending among other factors on the signal, of course) and the rest of coefficients are going to be zero-ed by the quantizer, if we apply a entropy compressor in the transform domain, the compression ratio compared to not using any transform should increase if we comparee such ratio to using the same entropy method applied directly over the RGB domain.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
!ln -sf ~/quantization/deadzone_quantizer.py .
!ln -sf ~/quantization/midtread_quantizer.py .
!ln -sf ~/quantization/midrise_quantizer.py .
!ln -sf ~/quantization/distortion.py .
!ln -sf ~/quantization/information.py .
!ln -sf ~/MRVC/src/color_DCT.py .
!ln -sf ~/MRVC/src/debug.py .
!ln -sf ~/MRVC/src/image_3.py .
!ln -sf ~/MRVC/src/image_1.py .
!ln -sf ../common.py .
import deadzone_quantizer as deadzone
import midtread_quantizer as midtread
import midrise_quantizer as midrise
import color_DCT
import distortion
import information
import image_3 as RGB_image
import image_1 as gray_image
import colored
import common

## Configuration

In [ ]:
# Prefix of the RGB image to be quantized.

home = os.environ["HOME"]
fn = home + "/MRVC/sequences/lena_color/"
#fn = home + "/MRVC/sequences/stockholm/"
image_dtype = np.uint8 # For 8 bpp/component images
#image_dtype = np.uint16 # For 16 bpp/component images

DCT_components = ['0', '1', '2']

# Number of quantization steps.
N_Q_steps = 8
#Q_steps = [256*i/N_Q_steps for i in range(N_Q_steps + 1, 0, -1)]
Q_steps = [2**i for i in range(8, 0, -1)]
print(Q_steps)

#quantizer = midtread
quantizer = deadzone
#quantizer = midrise

## Read the image and show it

In [ ]:
RGB_img = RGB_image.read(fn).astype(image_dtype)
common.show(RGB_img, fn + "000.png")

In [ ]:
RGB_img.shape

In [ ]:
RGB_img.dtype

## (RGB -> ColorDCT) transform of the image

In [ ]:
import numpy as np
from scipy.fftpack import dct, idct

DCT_type = 3
norm = "ortho" # Orthonormal: orthogonal + unitary (unit gain in both directions of the transform)
#norm = None

def color_DCT(RGB_img):
    DCT_img = np.empty_like(RGB_img).astype(np.float)
    for y in range(RGB_img.shape[0]):
        for x in range(RGB_img.shape[1]):
            DCT_img[y, x] = dct(RGB_img[y, x], type=DCT_type, norm=norm)
    return DCT_img

def color_IDCT(DCT_img):
    RGB_img = np.empty_like(DCT_img)#.astype(image_dtype)
    for y in range(DCT_img.shape[0]):
        for x in range(DCT_img.shape[1]):
            RGB_img[y, x] = idct(DCT_img[y, x], type=DCT_type, norm=norm)
    return RGB_img

In [ ]:
#DCT_img = color_DCT(RGB_img)
DCT_img = color_DCT.from_RGB(RGB_img)
print(DCT_img.dtype)

In [ ]:
common.show_gray(DCT_img[..., 0], fn + "000 (DCT0)")

In [ ]:
common.show_gray(DCT_img[..., 1], fn + "000 (DCT1)")

In [ ]:
common.show_gray(DCT_img[..., 2], fn + "000 (DCT2)")

## Energy of the DCT components

In [ ]:
DCT0_avg_energy = information.average_energy(DCT_img[..., 0])
DCT1_avg_energy = information.average_energy(DCT_img[..., 1])
DCT2_avg_energy = information.average_energy(DCT_img[..., 2])
print(f"Average energy DCT_img[..., 0] = {int(DCT0_avg_energy)}")
print(f"Average energy DCT_img[..., 1] = {int(DCT1_avg_energy)}")
print(f"Average energy DCT_img[..., 2] = {int(DCT2_avg_energy)}")
total_DCT_avg_energy = DCT0_avg_energy + DCT1_avg_energy + DCT2_avg_energy
print(f"Total average energy (computed by adding the energies of the DCT coefficients {int(DCT0_avg_energy)} + {int(DCT1_avg_energy)} + {int(DCT2_avg_energy)} = {int(total_DCT_avg_energy)}")
print(f"Total RGB average energy (computed directly from the RGB image) = {int(information.average_energy(RGB_img)*3)}")

Therefore, the forward DCT is energy preserving (unitary).

In [ ]:
RGB_recons_img = color_DCT.to_RGB(DCT_img)
print(f"Total RGB average energy (computed from the reconstructed RGB image) = {int(information.average_energy(RGB_recons_img)*3)}")

And the same can be said of the backward transform.

## (RGB <-> ColorDCT) transform error

The DCT transform is irreversible. In general, only integer arithmetic operations guarantees reversibility.

In [ ]:
common.show(RGB_recons_img, fn + "000.png (DCT recons)")

In [ ]:
np.array_equal(RGB_img, RGB_recons_img)

In [ ]:
print(RGB_img.max(), RGB_img.min())

In [ ]:
print(RGB_recons_img.max(), RGB_recons_img.min())

In [ ]:
common.show(RGB_img - RGB_recons_img.astype(image_dtype), "Reconstruction error (rounding error) color-DCT")

## Relative gains of the synthesis filters

The synthesis filters gains are important because the quantization steps of each DCT component should be adjusted in order to effectively provide the desired number of [bins](http://www.winlab.rutgers.edu/~crose/322_html/quantization.pdf) (different dequantized values) in each component.

In [ ]:
def print_info(val):
    DCT0_delta = np.array([val, 0, 0])
    RGB_DCT0_delta = idct(DCT0_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT0_delta = information.energy(RGB_DCT0_delta)
    
    DCT1_delta = np.array([0, val, 0])
    RGB_DCT1_delta = idct(DCT1_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT1_delta = information.energy(RGB_DCT1_delta)
    
    DCT2_delta = np.array([0, 0, val])
    RGB_DCT2_delta = idct(DCT2_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT2_delta = information.energy(RGB_DCT2_delta)
    
    zero = np.array([0, 0, 0])
    RGB_zero = idct(zero, type=DCT_type, norm=norm)
    RGB_energy_zero = information.energy(RGB_zero)
    
    print(f"{val}^2 = {val*val}")
    
    print(f"Energy of {DCT0_delta} in the RGB domain ({RGB_DCT0_delta}) = {RGB_energy_DCT0_delta}")
    print(f"Energy of {DCT1_delta} in the RGB domain ({RGB_DCT1_delta}) = {RGB_energy_DCT1_delta}")
    print(f"Energy of {DCT2_delta} in the RGB domain ({RGB_DCT2_delta}) = {RGB_energy_DCT2_delta}")
    print(f"Energy of {zero} in the RGB domain ({RGB_zero}) = {RGB_energy_zero}")
    
    max_ = max(RGB_energy_DCT0_delta, RGB_energy_DCT1_delta, RGB_energy_DCT2_delta)
    DCT0_relative_gain = RGB_energy_DCT0_delta / max_
    DCT1_relative_gain = RGB_energy_DCT1_delta / max_
    DCT2_relative_gain = RGB_energy_DCT2_delta / max_
    print(f"Relative gain of DCT0 component = {DCT0_relative_gain}")
    print(f"Relative gain of DCT1 component = {DCT1_relative_gain}")
    print(f"Relative gain of DCT2 component = {DCT2_relative_gain}")
    
print_info(255)
print()
print_info(1)
print()
print_info(0)

The gain of each DCT inverse filter is 1. Therefore, the optimal quantization pattern is $\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$.

## Amplitude shift in the DCT domain

To decide how to quantize, it is necessary to known how the amplitudes of the original image are *translated* to the transform domain. A good choice to find out this is to transform noise. In our case, lets use a random image with ([normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html)) [Gaussian noise](https://en.wikipedia.org/wiki/Gaussian_noise) with mean 128, DCT-it, and check where the transformed noise has its mean in each component. Notice that, by definition, the noise cannot be decorrelated by transforms, and therefore, the noise is simply transfered to the transform domain. Thus, depending on where the transformed noise has mean, we can decide if the signal must be shifted before or after the transform. Notice that the input signal to a dead-zone quantizer must have 0 mean.

In [ ]:
# loc = mean, scale=standard deviation, size=number of samples
RGB_noise = np.random.normal(loc=0, scale=10, size=512*512*3).reshape(512,512,3)#.astype(image_dtype)

In [ ]:
common.show(RGB_noise, "Gaussian RGB noise")

In [ ]:
DCT_noise = color_DCT.from_RGB(RGB_noise)

In [ ]:
common.show_gray(DCT_noise[..., 0], "Gaussian DCT0 noise")

In [ ]:
common.show_gray(DCT_noise[..., 1], "Gaussian DCT1 noise")

In [ ]:
common.show_gray(DCT_noise[..., 2], "Gaussian DCT2 noise")

The DCT does not modify the mean of the signal when it has 0 mean.

## Simple quantization in the DCT domain ($\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$)
Notice that, because the transform is orthogonal, the distortion can be measured in both, the RGB and the DCT domains, and that when the quantization steps are high enough, a 128-mean reconstructed RGB image should be generated.

In [ ]:
def DCT_same_delta_RD_curve_DCTdistortion(RGB_img, Q_steps, quantizer):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    DCT_img = color_DCT.from_RGB(RGB_img)#.astype(np.int16)
    points = []
    #common.show(DCT_img,'')
    for Q_step in Q_steps:
        DCT_y, k = quantizer.quan_dequan(DCT_img, Q_step)
        #common.show(DCT_y)
        print(np.max(DCT_y), np.min(DCT_y), np.average(DCT_y), DCT_y.dtype)
        BPP = common.bits_per_color_pixel((k + 128).astype(np.uint8), str(Q_step) + '_')
        print("Used quantization indexes:", np.unique((k + 128).astype(np.uint8)))
        RMSE = distortion.RMSE(DCT_img, DCT_y) # Uncomment this line for measuring the distortion in the DCT domain
        points.append((BPP, RMSE))
        print(f"q_step={Q_step}, rate={BPP} bits/pixel, distortion={RMSE}")
    return points

DCT_same_delta_RD_points_DCTdistortion = DCT_same_delta_RD_curve_DCTdistortion(RGB_img, Q_steps, quantizer)

In [ ]:
def DCT_same_delta_RD_curve_RGBdistortion(RGB_img, Q_steps, quantizer):
    RGB_img = RGB_img.astype(float)
    avg0 = np.average(RGB_img[..., 0])
    avg1 = np.average(RGB_img[..., 1])
    avg2 = np.average(RGB_img[..., 2])
    RGB_img[..., 0] -= avg0
    RGB_img[..., 1] -= avg1
    RGB_img[..., 2] -= avg2
    DCT_img = color_DCT.from_RGB(RGB_img)#.astype(np.int16)
    points = []
    common.show(RGB_img,'')
    #DCT_img -= (215-128)
    for Q_step in Q_steps:
        DCT_y, k = quantizer.quan_dequan(DCT_img, Q_step)
        #common.show(DCT_y)
        print(np.max(DCT_y), np.min(DCT_y), np.average(DCT_y), DCT_y.dtype)
        #common.show(DCT_y, '')
        BPP = common.bits_per_color_pixel((k + 128).astype(np.uint8), str(Q_step) + '_')
        print("Used quantization indexes:", np.unique((k + 128).astype(np.uint8)))
        #RGB_y = color_IDCT(DCT_y + (215-128))                    # Uncomment these lines for measuring
        RGB_y = color_DCT.to_RGB(DCT_y) # Uncomment these lines for measuring
        #RGB_y[..., 0] += avg0
        #RGB_y[..., 1] += avg1
        #RGB_y[..., 2] += avg2
        RMSE = distortion.RMSE(RGB_img, RGB_y) # the distortion in the RGB domain.
        common.show(RGB_y, '')
        #_distortion = distortion.RMSE(DCT_img, DCT_y) # Uncomment this line for measuring the distortion in the DCT domain
        points.append((BPP, RMSE))
        print(f"q_step={Q_step}, rate={BPP} bits/pixel, distortion={RMSE}")
    return points

DCT_same_delta_RD_points_RGBdistortion = DCT_same_delta_RD_curve_RGBdistortion(RGB_img, Q_steps, quantizer)

## Let's see the RD curve

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points_DCTdistortion), c='r', marker='.', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$ (Distortion in DCT domain)", linestyle="dashed")
pylab.plot(*zip(*DCT_same_delta_RD_points_RGBdistortion), c='b', marker='.', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$ (Distortion in RGB domain)", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

The distortion can be measured in both domains, DCT and RGB. This is true because the DCT is orthogonal.

## Comparison with quantizing directly in the RGB domain

In [ ]:
RGB_points = []
with open(f'../05-RGB_compression/RGB.txt', 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_points.append((float(rate), float(_distortion)))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RGB_points), c='b', marker='o', label="Quantization in the RGB domain", linestyle="dashed")
pylab.plot(*zip(*DCT_same_delta_RD_points_DCTdistortion), c='r', marker='x', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

The color-DCT removes the color redundancy, generating three components that are more compressible than the original RGB channels. This effect is more significant a low bit-rates. 

## Optimal RD curve (measuring distortion in the color-DCT domain)
Works because the color-DCT is orthogonal.

As we did in the RGB domain, we will try to apply RDO in the color-DCT domain, to find out if a better RD curve than using $\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$ can be obtained. However, as also happened with the RGB domain, the DCT components are independent (the DCT is orthogonal) and additive. Therefore, apart from generating a RD curve with more points, we should not expect to improve it.

Notice that the distortion can be measured in both domains, color-DCT and RGB, because the transform is orthogonal.

Therefore, such subbands should be encoded first in a progressive representation of the image (to obtain a RD curve) in the transform domain.

To find such progressive quantization steps patterns (that in the case of the color-DCT has 3 dimensions, one for each color-DCT component) we will use (again, as we did to compress the images in the RGB domain) a RDO (Rate/Distortion Optimization) algorithm based on sorting the patterns by the slopes of the RD points in the corresponding affected subbands (remember that between two consecutive patterns, only one quantization step can replaced by the next smaller value).  

The method `DCT_same_delta_RD_curve(img, Q_steps, quantizer)` generates a RD curve where each point is the result of using $\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$. However, a better (at least with more points) RD curve can be generated with:

1. Convert the image from RGB to DCT{0,1,2}.
2. The RD curve of each DCT channel is computed, for a number of quantization steps, measuring the distortion in the RGB domain. We will quantize one component at each iteration and the rest of components will be unquantized. This is necessary to ensure that the low-pass component (DCT0) is always considered in each reconstruction. Otherwise, the distortion (at least using the RMSE) will not be estimated correctly.
3. Compute the slope of each segment of the RD curve. Except for the most left point, the slopes are computed as the average between the slopes of the straight lines that connect to the corresponding point.
4. For each quantization step, sort the RD points by their slope.
5. Recompute the optimal RD curve using the quantization steps provided by the sorted RD points.

## RD curve of each color-DCT component
We consider only the rate and the distortion of the component.

In [ ]:
def color_DCT_RD_curve_per_component(RGB_img, Q_steps, Q, components):
    N_components = len(components)
    RGB_img = RGB_img.astype(float)
    for c in range(N_components):
        avg = np.average(RGB_img[..., c])
        RGB_img[..., c] -= avg
        print(f"channel={c} average={avg}")
    DCT_img = color_DCT.from_RGB(RGB_img)
    DCT_img_copy = DCT_img.copy()
    RD_points = []
    for c in range(N_components):
        RD_points.append([])
    for Q_step in Q_steps:
        DCT_k = Q.quantize(DCT_img, Q_step)
        for component_index in range(N_components):
            component_name = components[component_index]
            DCT_y = np.zeros_like(DCT_img)
            DCT_y = DCT_img_copy.copy()
            DCT_y[..., component_index] = Q.dequantize(DCT_k[..., component_index], Q_step)
            #print(Q_step, DCT_k[..., component_index].max(), DCT_k[..., component_index].min(), information.entropy(DCT_k[..., component_index]))
            BPP = common.bits_per_gray_pixel(DCT_k[..., component_index] + 128, str(Q_step) + '_' + component_name + '_' + str(components[component_index]))
            print("Used quantization indexes:", np.unique(DCT_k[..., component_index] + 128))
            #RGB_y = color_DCT.to_RGB(DCT_y)
            #_distortion = distortion.RMSE(RGB_img, RGB_y)
            RMSE = distortion.RMSE(DCT_img[..., component_index], DCT_y[..., component_index])
            #common.show(RGB_y, components[component_index] + ' ' + str(Q_step))
            RD_points[component_index].append((BPP, RMSE, component_name, Q_step))
            print(f"component_index={components[component_index]} q_step={Q_step}, rate={BPP} bits/pixel, distortion={RMSE}")
    return RD_points
DCT_RD_curve_per_component = color_DCT_RD_curve_per_component(RGB_img, Q_steps, quantizer, DCT_components)

In [ ]:
#for c in range(3):
#    DCT_RD_curve_per_component[c] = [DCT_RD_curve_per_component[c][0]] + DCT_RD_curve_per_component[c]
#    DCT_RD_curve_per_component[c][0] = (0, -DCT_RD_curve_per_component[c][0][1], f'{c}', 256)

In [ ]:
DCT_RD_curve_per_component

In [ ]:
def filter_points(points):
    filtered_points = []
    points_iterator = iter(points)
    prev = next(points_iterator)
    for curr in points_iterator:
        if prev[0] >= curr[0]:
            print(f"deleted {prev}")
        else:
            filtered_points.append(prev)
        prev = curr
    filtered_points.append(prev)
    return filtered_points

for c in range(3):
    DCT_RD_curve_per_component[c] = filter_points(DCT_RD_curve_per_component[c])

In [ ]:
DCT_RD_curve_per_component

## Display the curve of each component

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0][1:]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1][1:]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2][1:]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE in RGB domain")
pylab.legend(loc='upper right')
pylab.show()

Each curve shows the impact of variying the quantization step, in the corresponding component. Moreover, since the color-DCT is orthonormal, the gain of each component is 1. Therefore, the relative position of the curves shows the contribution (remember that if we are using the RMSE as the distortion metric and for this reason, me are basically measuring energy) of each component to the quality of the reconstructed image. Considering all this information, we should use the following progression of quantization steps patterns:
1. `[256, 256, 256]` -> Constant gray image.
2. `[128, 256, 256]` -> Grayscale image.
3. `[ 64, 256, 256]` -> A higher quality image.
4. `[128, 128, 128]` -> Same color image.
5. `[ 64, 128, 128]` -> A higher quality color image.
6. `[ 32, 128, 128]` -> ...
7. `[ 16, 128, 128]` -> ...

In [ ]:
def color_DCT_optimal_curve(RGB_img, RD_points_per_component):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image (the average of each component is 0).
    DCT_img = color_DCT.from_RGB(RGB_img)
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    gains = [RD_points_per_component[0][0][1],
             RD_points_per_component[1][0][1],
             RD_points_per_component[2][0][1]]
    max_gain = max(gains)
    where = np.argmax(gains)
    Q_steps_indexes = [0, 0, 0]
    prev_Q_steps_indexes = [0, 0, 0]
    #Q_steps_indexes[where] += 1
    while Q_steps_per_component != [4, 4, 4]:
        Q_steps_per_component = [256, 256, 256]
        for i in range(3):
            Q_steps_per_component[i] >>= Q_steps_indexes[i]
        print(f"{Q_steps_indexes} Quantization steps pattern: {Q_steps_per_component}")
        for component_index, Q_step in zip([0, 1, 2], Q_steps_per_component):
            DCT_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(DCT_img[..., component_index], Q_step)
        BPP = common.bits_per_color_pixel((k + 128).astype(np.uint8), str(Q_steps_per_component) + '_')
        print("Used quantization indexes:", np.unique((k + 128).astype(np.uint8)))
        RMSE = distortion.RMSE(DCT_img, DCT_y)
        points.append((BPP, RMSE))
        print(f"rate={BPP} bits/pixel, distortion={RMSE}")
        Q_steps_indexes[where] += 1
        slopes = []
        print("Considering")
        for c in range(3):
            if len(RD_points_per_component[c]) > (Q_steps_indexes[c] + 1):
                delta_BPP = RD_points_per_component[c][Q_steps_indexes[c] + 1][0] - RD_points_per_component[c][Q_steps_indexes[c]][0]
                delta_RMSE = RD_points_per_component[c][Q_steps_indexes[c]][1] - RD_points_per_component[c][Q_steps_indexes[c] + 1][1]            #print(prev_Q_steps_indexes, Q_steps_indexes, RD_points_per_component[c][prev_Q_steps_indexes[0]], RD_points_per_component[c][Q_steps_indexes[0]])
                print(f"delta_BPP={delta_BPP}")
                if delta_BPP > 0:
                    slope = delta_RMSE/delta_BPP
                else:
                    slope = 0
                slopes.append(slope)
            else:
                slopes.append(0)
        print(f"slopes={slopes}")
        where = np.argmax(slopes)
        prev_Q_steps_indexes = Q_steps_indexes.copy()
    return points

DCT_optimal_RD_points = color_DCT_optimal_curve(RGB_img, DCT_RD_curve_per_component)
DCT_optimal_RD_points

## Compute the slope of each point

In [ ]:
def compute_slopes(RD_points):
    counter = 0
    RD_slopes = []
    points_iterator = iter(RD_points)
    next(points_iterator)
    for i in points_iterator:
        BPP = i[0] # Rate 
        delta_BPP = BPP - RD_points[counter][0]
        RMSE = i[1] # Distortion
        delta_RMSE = RMSE - RD_points[counter][1] 
        if delta_BPP > 0:
            slope = abs(delta_RMSE/delta_BPP)
        else:
            slope = 0
        component = i[2]
        Q_step = i[3]
        print((slope, i), delta_RMSE, delta_BPP)
        RD_slopes.append((slope, component, Q_step))
        counter += 1
    return RD_slopes

In [ ]:
#DCT0_slopes = common.compute_slopes(DCT_RD_curve_per_component[0])
#DCT1_slopes = common.compute_slopes(DCT_RD_curve_per_component[1])
#DCT2_slopes = common.compute_slopes(DCT_RD_curve_per_component[2])
DCT0_slopes = compute_slopes(DCT_RD_curve_per_component[0])
DCT1_slopes = compute_slopes(DCT_RD_curve_per_component[1])
DCT2_slopes = compute_slopes(DCT_RD_curve_per_component[2])

In [ ]:
DCT0_slopes

In [ ]:
DCT1_slopes

In [ ]:
DCT2_slopes

## Filter the curves
Remove those RD points that do not belong to the convex-hull.

In [ ]:
DCT0_slopes = common.filter_slopes(DCT0_slopes)
DCT0_slopes

In [ ]:
DCT1_slopes = common.filter_slopes(DCT1_slopes)
DCT1_slopes

In [ ]:
DCT2_slopes = common.filter_slopes(DCT2_slopes)
DCT2_slopes = common.filter_slopes(DCT2_slopes)
DCT2_slopes

## Sort the slopes at each quantization step
Notice that the TPs (Truncation Points) generated in a component must be used in order.

In [ ]:
all_slopes = DCT0_slopes + DCT1_slopes + DCT2_slopes
sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]
sorted_slopes

## Compute the optimal RD curve
And finally, let's compute the RD curve (remember that the previous points are only an estimation of the order in which the quantization steps should be increased in each component to build the RD curve, not the real RD curve that measures the distortion in the RGB domain).

In [ ]:
def DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, components):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image (the average of each component is 0).
    DCT_img = color_DCT.from_RGB(RGB_img)
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    for i in sorted_slopes:
        print(i)
        component, Q_step = i[1], i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        for c, Q_step in zip(components, Q_steps_per_component):
            component_index = components.index(c)
            DCT_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(DCT_img[..., component_index], Q_step)
        rate = common.bits_per_color_pixel((k + 128).astype(np.uint8), str(Q_steps_per_component) + '_')
        print("Used quantization indexes:", np.unique((k + 128).astype(np.uint8)))
        #y_RGB = YCrCb.to_RGB(y + 128)
        #RGB_y = color_DCT.to_RGB(DCT_y)               # Uncomment to compute distortion
        #_distortion = distortion.RMSE(RGB_img, RGB_y)  # in the RGB domain.
        _distortion = distortion.RMSE(DCT_img, DCT_y)  # Uncomment to compute distortion in the DCT domain.
        #common.show(y_RGB, f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points



DCT_optimal_RD_points = DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, DCT_components)
DCT_optimal_RD_points

## Compare the curves

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points_RGBdistortion), c='b', marker='o', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*DCT_optimal_RD_points), c='r', marker='x', label="Optimal", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open('color_DCT.txt', 'w') as f:
    for item in DCT_optimal_RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

## Conclusión
The performance of the simple quantization solution ($\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$) is very close to the optimal one. This is normal because the DCT is orthogonal.

## Optimal RD curve (measuring distortion in the RGB domain)
Useful when the transform is not orthogonal.

## RD curve of each DCT component

Notice that the distortion has been measured in the RGB domain and that the components are not 

In [ ]:
def color_DCT_RD_curve_per_component_(RGB_img, Q_steps, Q, components):
    N_components = len(components)
    RGB_img = RGB_img.astype(float)
    for c in range(N_components):
        avg = np.average(RGB_img[..., c])
        RGB_img[..., c] -= avg
        print(f"channel={c} average={avg}")
    DCT_img = color_DCT.from_RGB(RGB_img)
    DCT_img_copy = DCT_img.copy()
    RD_points = []
    for c in range(N_components):
        RD_points.append([])
    for Q_step in Q_steps:
        DCT_k = Q.quantize(DCT_img, Q_step)
        for component_index in range(N_components):
            component_name = components[component_index]
            DCT_y = np.zeros_like(DCT_img)
            DCT_y = DCT_img_copy.copy()
            DCT_y[..., component_index] = Q.dequantize(DCT_k[..., component_index], Q_step)
            #print(Q_step, DCT_k[..., component_index].max(), DCT_k[..., component_index].min(), information.entropy(DCT_k[..., component_index]))
            rate = common.bits_per_gray_pixel(DCT_k[..., component_index], str(Q_step) + '_' + component_name + '_' + str(components[component_index]))
            RGB_y = color_DCT.to_RGB(DCT_y)
            _distortion = distortion.RMSE(RGB_img, RGB_y)
            #common.show(RGB_y, components[component_index] + ' ' + str(Q_step))
            RD_points[component_index].append((rate, _distortion, component_name, Q_step))
            print(f"component_index={components[component_index]} q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return RD_points
           
def color_DCT_RD_curve_per_component(RGB_img, Q_steps, Q, components):
    N_components = len(components)
    RGB_img = RGB_img.astype(float)
    for c in range(N_components):
        avg = np.average(RGB_img[..., c])
        RGB_img[..., c] -= avg
        print(f"channel={c} average={avg}")
    DCT_img = color_DCT.from_RGB(RGB_img)
    DCT_img_copy = DCT_img.copy()
    RD_points = []
    for c in range(N_components):
        RD_points.append([(0, 100000, c, 512)])
        #RD_points.append([])
    for Q_step in Q_steps:
        for component_index in range(N_components):
            component_name = components[component_index]
            #DCT_k = np.zeros_like(DCT_img)
            DCT_k = DCT_img_copy.copy()
            DCT_y = DCT_img_copy.copy()
            DCT_k[..., component_index] = Q.quantize(DCT_img[..., component_index], Q_step)
            DCT_y[..., component_index] = Q.dequantize(DCT_k[..., component_index], Q_step)
            #print(Q_step, DCT_k[..., component_index].max(), DCT_k[..., component_index].min(), information.entropy(DCT_k[..., component_index]))
            BPP = common.bits_per_gray_pixel(DCT_k[..., component_index].astype(np.uint8), str(Q_step) + '_' + component_name + '_' + str(components[component_index]))
            RGB_y = color_DCT.to_RGB(DCT_y)
            RMSE = distortion.RMSE(RGB_img, RGB_y)
            #common.show(RGB_y, components[component_index] + ' ' + str(Q_step))
            RD_points[component_index].append((BPP, RMSE, component_name, Q_step))
            print(f"component_index={components[component_index]} q_step={Q_step}, rate={BPP} bits/pixel, distortion={RMSE}")
    return RD_points

DCT_RD_curve_per_component = color_DCT_RD_curve_per_component(RGB_img, Q_steps, quantizer, DCT_components)

In [ ]:
DCT_RD_curve_per_component

## Display the curve of each component

In [ ]:
pylab.figure(dpi=150)
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0][1:]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1][1:]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2][1:]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE in RGB domain")
pylab.legend(loc='upper right')
pylab.show()

Notice that this plot represents the impact in the RGB domain of quantizing each component, keeping the rest unquantized. Therefore, the most important component from a RD point of view is DCT0.

## Compute the slope of each point

In [ ]:
def compute_slopes(RD_points):
    counter = 0
    RD_slopes = []
    points_iterator = iter(RD_points)
    next(points_iterator)
    for i in points_iterator:
        BPP = i[0] # Rate 
        delta_BPP = BPP - RD_points[counter][0]
        RMSE = i[1] # Distortion
        delta_RMSE = RMSE - RD_points[counter][1] 
        if delta_BPP > 0:
            slope = abs(delta_RMSE/delta_BPP)
        else:
            slope = 0
        component = i[2]
        Q_step = i[3]
        print((slope, i), delta_RMSE, delta_BPP)
        RD_slopes.append((slope, component, Q_step))
        counter += 1
    return RD_slopes

In [ ]:
#DCT0_slopes = common.compute_slopes(DCT_RD_curve_per_component[0])
#DCT1_slopes = common.compute_slopes(DCT_RD_curve_per_component[1])
#DCT2_slopes = common.compute_slopes(DCT_RD_curve_per_component[2])
DCT0_slopes = compute_slopes(DCT_RD_curve_per_component[0])
DCT1_slopes = compute_slopes(DCT_RD_curve_per_component[1])
DCT2_slopes = compute_slopes(DCT_RD_curve_per_component[2])

In [ ]:
DCT0_slopes

In [ ]:
DCT1_slopes

In [ ]:
DCT2_slopes

## Filter the curves
Remove those RD points that do not belong to the convex-hull.

In [ ]:
DCT0_slopes = common.filter_slopes(DCT0_slopes)
DCT0_slopes

In [ ]:
DCT1_slopes = common.filter_slopes(DCT1_slopes)
DCT1_slopes

In [ ]:
DCT2_slopes = common.filter_slopes(DCT2_slopes)
DCT2_slopes = common.filter_slopes(DCT2_slopes)
DCT2_slopes

## Sort the slopes at each quantization step
Notice that the TPs (Truncation Points) generated in a component must be used in order.

In [ ]:
all_slopes = DCT0_slopes + DCT1_slopes + DCT2_slopes
sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]
sorted_slopes

## Compute the optimal RD curve
And finally, let's compute the RD curve (remember that the previous points are only an estimation of the order in which the quantization steps should be increased in each component to build the RD curve, not the real RD curve that measures the distortion in the RGB domain).

In [ ]:
def DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, components):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image (the average of each component is 0).
    DCT_img = color_DCT.from_RGB(RGB_img)
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    for i in sorted_slopes:
        print(i)
        component, Q_step = i[1], i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        for c, Q_step in zip(components, Q_steps_per_component):
            component_index = components.index(c)
            DCT_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(DCT_img[..., component_index], Q_step)
        rate = common.bits_per_color_pixel(k.astype(np.uint8), str(Q_steps_per_component) + '_')
        #y_RGB = YCrCb.to_RGB(y + 128)
        #RGB_y = color_DCT.to_RGB(DCT_y)               # Uncomment to compute distortion
        #_distortion = distortion.RMSE(RGB_img, RGB_y)  # in the RGB domain.
        _distortion = distortion.RMSE(DCT_img, DCT_y)  # Uncomment to compute distortion in the DCT domain.
        #common.show(y_RGB, f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

DCT_optimal_RD_points = DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, DCT_components)
DCT_optimal_RD_points

## Compare the curves

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points_RGBdistortion), c='b', marker='o', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*DCT_optimal_RD_points), c='r', marker='x', label="Optimal", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open('color_DCT.txt', 'w') as f:
    for item in DCT_optimal_RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

## Conclusión
The performance of the simple quantization solution ($\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$) is very close to the optimal one. This is normal because the DCT is orthogonal.

## Compare to quantizing directly in the RGB domain

In [ ]:
RGB_points = []
with open(f'../05-RGB_compression/RGB.txt', 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_points.append((float(rate), float(_distortion)))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RGB_points), c='b', marker='o', label="Quantization in the RGB domain", linestyle="dashed")
pylab.plot(*zip(*DCT_optimal_RD_points), c='r', marker='x', label="Quantization in the color-DCT domain", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()